# 6. Análise dos dados - Parte 1

In [1]:
import polars as pl
df = pl.read_csv('base_fapesp_tratada.csv')
df.columns

['N. Processo',
 'Título (Português)',
 'Título (Inglês)',
 'Beneficiário',
 'Instituição',
 'Cidade Instituição',
 'Instituição Parceira',
 'Pesquisador Responsável',
 'Local de Pesquisa',
 'Modalidade de apoio',
 'Grande Área do Conhecimento',
 'Área do Conhecimento',
 'Subárea do Conhecimento',
 'Assuntos',
 'Data de Início',
 'Data de Término',
 'Acordo(s)/Convênio(s) de Cooperação com a FAPESP',
 'Instituições no Exterior',
 'Pais de Origem',
 'Pesquisador responsável no exterior',
 'Resumo (Português)',
 'Resumo (Inglês)',
 'Processos Vinculados']

Vamos começar então a responder a perguntas que fizemos no tópico anterior:

## Quais são as cidades que mais recebem financiamento da FAPESP?

Para responder a essa pergunta, como vimos, a coluna relevante é Cidade Instituição. Para responder essa pergunta, é extremamente simples, basta contarmos os valores desta coluna.

In [2]:
df['Cidade Instituição'].value_counts(sort=True)

Cidade Instituição,count
str,u32
"""São Paulo""",49279
"""Campinas""",22621
"""São Carlos""",14888
"""Ribeirão Preto""",10875
"""Botucatu""",6299
…,…
"""Guarujá""",1
"""Espírito Santo do Pinhal""",1
"""Agudos""",1


Podemos focar nas cidades que mais recebem financiamento:

In [3]:
df['Cidade Instituição'].value_counts(sort=True).head(10)

Cidade Instituição,count
str,u32
"""São Paulo""",49279
"""Campinas""",22621
"""São Carlos""",14888
"""Ribeirão Preto""",10875
"""Botucatu""",6299
"""Piracicaba""",5966
null,5703
"""Araraquara""",4316
"""Bauru""",4003


Ou salvar o resultado completo como uma planilha no excel para utilizar esses dados em nosso artigo científico.

In [10]:
df['Cidade Instituição'].value_counts(sort=True).write_excel('fapesp_cidades.xlsx')

Bastante simples, certo? Isso não é tão diferente de gerar uma tabela dinâmica no excel, mas o polars te permite maior precisão e velocidade na manipulação dos dados, além de tornar possível compartilhar exatamente o que você fez para gerar cada conjunto de dados incluído no seu artigo.

## Quais são as principais áreas do conhecimento e como elas se dividem entre as grandes áreas?
Para responder essas perguntas, duas colunas principais importam: Grande Área do Conhecimento e Área do Conhecimento.

In [5]:
df['Grande Área do Conhecimento'].value_counts(sort=True)

Grande Área do Conhecimento,count
str,u32
"""Ciências Biológicas""",34201
"""Ciências da Saúde""",26943
"""Ciências Exatas e da Terra""",26081
"""Ciências Agrárias""",19452
"""Engenharias""",17002
"""Ciências Humanas""",16937
"""Ciências Sociais Aplicadas""",7284
"""Linguística, Letras e Artes""",6190
"""Interdisciplinar""",1677


In [6]:
df['Área do Conhecimento'].value_counts(sort=True)

Área do Conhecimento,count
str,u32
"""Medicina""",9868
"""Química""",7974
"""Odontologia""",7032
"""Medicina Veterinária""",6111
"""Física""",5580
…,…
"""Engenharia de Minas""",92
"""Demografia""",91
"""Teologia""",45


Analisar esses dados separadamente é fácil e nós já sabemos como fazer isso, mas e analisar as duas colunas em conjunto?

Para isso, precisamos agrupar os dados de uma coluna a partir dos dados da outra. Isso pode ser feito usando o `.group_by()`.

In [7]:
(
    df
    .group_by(['Grande Área do Conhecimento', 'Área do Conhecimento'])
    .agg(pl.len().alias('count'))
    .sort('count', descending=True)
    .head(10)
)

Grande Área do Conhecimento,Área do Conhecimento,count
str,str,u32
"""Ciências da Saúde""","""Medicina""",9868
"""Ciências Exatas e da Terra""","""Química""",7974
"""Ciências da Saúde""","""Odontologia""",7032
"""Ciências Agrárias""","""Medicina Veterinária""",6111
"""Ciências Exatas e da Terra""","""Física""",5579
"""Ciências Biológicas""","""Bioquímica""",5399
"""Ciências Agrárias""","""Agronomia""",5357
"""Ciências Biológicas""","""Genética""",4610
"""Ciências Exatas e da Terra""","""Ciência da Computação""",4593


Vamos entender esse código:

`(df`: Abrimos o parenteses para indicar que tudo que vem a seguir é uma só operação e declaramos que queremos trabalhar com `df`.

`.group_by(['Grande Área do Conhecimento', 'Área do Conhecimento'])`: agrupamos df por estas duas colunas.

`.agg(pl.len().alias('count'))`: contamos qual o tamanho (length, por isso len()) de cada subconjunto dessas duas e salvamos esse resultado em uma coluna que irá se chamar "count".

`.head(10)`: selecionamos apenas os 10 primeiros resultados.

Com essa operação, pudemos ver que as áreas do conhecimento que mais recebem bolsas da fapesp são principalmente áreas ligadas a ciências da saúde e biológicas.

Mas e se, por exemplo, quisessemos na verdade todos os resultados e que, para cada grande área, tivessemos o ranking do maior ao menor?

In [8]:
(
    df
    .group_by(['Grande Área do Conhecimento', 'Área do Conhecimento'])
    .agg(pl.len().alias('count'))
    .sort(['Grande Área do Conhecimento', 'count'], descending=[False, True])
)

Grande Área do Conhecimento,Área do Conhecimento,count
str,str,u32
"""Ciências Agrárias""","""Medicina Veterinária""",6111
"""Ciências Agrárias""","""Agronomia""",5357
"""Ciências Agrárias""","""Zootecnia""",2980
"""Ciências Agrárias""","""Ciência e Tecnologia de Alimen…",2773
"""Ciências Agrárias""","""Recursos Pesqueiros e Engenhar…",848
…,…,…
"""Interdisciplinar""",null,1676
"""Interdisciplinar""","""Física""",1
"""Linguística, Letras e Artes""","""Letras""",2352


Para isso, tiramos o `.head(10)` e acrescentamos:

`.sort(['Grande Área do Conhecimento', 'count'], descending=[False, True])` : ordenamos Grande Área do Conhecimento em ordem alfabética e, dentro de cada grande área, mostramos do maior para o menor a partir da contagem.




"Mas poxa, Bruno, na verdade eu só me interesso por ciências sociais aplicadas e elas nem aparecem direito nesses rankings."

Podemos olhar especificamente para esse subconjunto de casos:

In [9]:
(
    df
    .filter(pl.col('Grande Área do Conhecimento') == 'Ciências Sociais Aplicadas')
    ['Área do Conhecimento']
    .value_counts(sort=True)
    .head(10)
)

Área do Conhecimento,count
str,u32
"""Arquitetura e Urbanismo""",2086
"""Direito""",1236
"""Economia""",1019
"""Comunicação""",990
"""Administração""",504
"""Planejamento Urbano e Regional""",474
"""Ciência da Informação""",400
"""Desenho Industrial""",189
"""Serviço Social""",144


Vejamos, depois de abrir parenteses, nós:

`df`: indicamos que estamos trabalhando com `df`.

`.filter(pl.col('Grande Área do Conhecimento') == 'Ciências Sociais Aplicadas')`: realizamos um filtro para selecionar apenas as linhas de `df` cujo conteúdo na coluna 'Grande Área do Conhecimento' é 'Ciências Sociais Aplicadas'.  
OBS: note que foi utilizado "==" e não apenas "=". No python, "==" faz uma comparação - é igual ou não é? -, enquanto o "=" faz uma atribuição - a partir de agora, isso É isso.

`['Área do Conhecimento']`: selecionamos a coluna que queremos observar.

`.value_counts(sort=True)`: contamos os elementos dessa coluna e os ordenamos.

`.head(10)`: identificamos os 10 primeiros itens resultantes da operação.

Eu particularmente não esperaria que arquitetura e urbanismo estivesse a frente de direito e economia. Temos já aí um resultado potencialmente interessante.